# 351 Android unlock patterns
### Overview
Most Android phones feature a security mechanism known as a lock pattern. It is a 3×3 grid of dots, where unmarked dots can be connected consecutively to create a pattern. More formally, from any dot in the grid, we can make 2 types of moves:

- Single-step move: This involves connecting two dots directly.


![image.png](https://assets.leetcode.com/users/images/457b8cdc-781a-4ee7-ac46-9da0f4669aaf_1723094355.9709423.png)
![image.png](https://assets.leetcode.com/users/images/6675f134-5df0-47ea-b9a8-c365f3fa44ca_1723094421.8091044.png)



## Approach 1: Backtracking
### Intuition
One approach to solving this problem involves generating all possible patterns and counting those that meet our specified conditions. Let's explore this method with a slight optimization.

First, we'll create two arrays to represent each type of possible move: `SINGLE_STEP_MOVES` and `SKIP_DOT_MOVES`.

To find each pattern, we'll employ a recursive function countPatternsFromDot which explores all possible moves from the current dot under examination. We'll keep track of the dots explored at each step and start counting once the number of dots exceeds m. The recursion will continue until the current pattern reaches n dots, at which point it terminates. This function will successfully return all possible patterns from any given starting point.

However, we can often predict whether a path will yield a satisfactory pattern well before reaching its end. For instance, if at any point in the pattern's development, all potential next moves violate the rules, we can conclude that any patterns stemming from this state will be invalid. At any step, if we determine that the current path will not lead to a valid solution, we can abandon it and return to a previous step that still shows promise. This technique, known as backtracking, helps optimize our approach. If you're unfamiliar with it, consider reviewing this LeetCode [Explore Card](https://leetcode.com/explore/learn/card/recursion-ii/472/backtracking/2654/) for a detailed explanation.

Since a valid pattern can start from any dot, we'll call countPatternsFromDot for each of the nine dots and aggregate the results. This total count represents the required number of unlock patterns.

### Algorithm
- Define constants:
    - `SINGLE_STEP_MOVES`: All possible adjacent and diagonal moves.
    - `SKIP_DOT_MOVES`: Moves that jump over a dot, requiring the middle dot to be visited.


Main method `numberOfPatterns`:
![image.png](https://assets.leetcode.com/users/images/4c278854-83c9-4adb-878a-2b5af638ae0c_1723094300.4215753.png)
![image.png](https://assets.leetcode.com/users/images/ae4ab456-f310-4924-82c0-f9856794d3ba_1723094500.5333056.png)

### Code


JAVA
```Java []
class Solution {

    // All possible single-step moves on the lock pattern grid
    // Each array represents a move as {row change, column change}
    private static final int[][] SINGLE_STEP_MOVES = {
        { 0, 1 },
        { 0, -1 },
        { 1, 0 },
        { -1, 0 }, // Adjacent moves (right, left, down, up)
        { 1, 1 },
        { -1, 1 },
        { 1, -1 },
        { -1, -1 }, // Diagonal moves
        { -2, 1 },
        { -2, -1 },
        { 2, 1 },
        { 2, -1 }, // Extended moves (knight-like moves)
        { 1, -2 },
        { -1, -2 },
        { 1, 2 },
        { -1, 2 },
    };

    // Moves that require a dot to be visited in between
    // These moves "jump" over a dot, which must have been previously visited
    private static final int[][] SKIP_DOT_MOVES = {
        { 0, 2 },
        { 0, -2 },
        { 2, 0 },
        { -2, 0 }, // Straight skip moves (e.g., 1 to 3, 4 to 6)
        { -2, -2 },
        { 2, 2 },
        { 2, -2 },
        { -2, 2 }, // Diagonal skip moves (e.g., 1 to 9, 3 to 7)
    };

    public int numberOfPatterns(int m, int n) {
        int totalPatterns = 0;
        // Start from each of the 9 dots on the grid
        for (int row = 0; row < 3; row++) {
            for (int col = 0; col < 3; col++) {
                boolean[][] visitedDots = new boolean[3][3];
                // Count patterns starting from this dot
                totalPatterns +=
                countPatternsFromDot(m, n, 1, row, col, visitedDots);
            }
        }
        return totalPatterns;
    }

    private int countPatternsFromDot(
        int m,
        int n,
        int currentLength,
        int currentRow,
        int currentCol,
        boolean[][] visitedDots
    ) {
        // Base case: if current pattern length exceeds n, stop exploring
        if (currentLength > n) {
            return 0;
        }

        int validPatterns = 0;
        // If current pattern length is within the valid range, count it
        if (currentLength >= m) validPatterns++;

        // Mark current dot as visited
        visitedDots[currentRow][currentCol] = true;

        // Explore all single-step moves
        for (int[] move : SINGLE_STEP_MOVES) {
            int newRow = currentRow + move[0];
            int newCol = currentCol + move[1];
            if (isValidMove(newRow, newCol, visitedDots)) {
                // Recursively count patterns from the new position
                validPatterns +=
                countPatternsFromDot(
                    m,
                    n,
                    currentLength + 1,
                    newRow,
                    newCol,
                    visitedDots
                );
            }
        }

        // Explore all skip-dot moves
        for (int[] move : SKIP_DOT_MOVES) {
            int newRow = currentRow + move[0];
            int newCol = currentCol + move[1];
            if (isValidMove(newRow, newCol, visitedDots)) {
                // Check if the middle dot has been visited
                int middleRow = currentRow + move[0] / 2;
                int middleCol = currentCol + move[1] / 2;
                if (visitedDots[middleRow][middleCol]) {
                    // If middle dot is visited, this move is valid
                    validPatterns +=
                    countPatternsFromDot(
                        m,
                        n,
                        currentLength + 1,
                        newRow,
                        newCol,
                        visitedDots
                    );
                }
            }
        }

        // Backtrack: unmark the current dot before returning
        visitedDots[currentRow][currentCol] = false;
        return validPatterns;
    }

    // Helper method to check if a move is valid
    private boolean isValidMove(int row, int col, boolean[][] visitedDots) {
        // A move is valid if it's within the grid and the dot hasn't been visited
        return (
            row >= 0 && col >= 0 && row < 3 && col < 3 && !visitedDots[row][col]
        );
    }
}

```
### Complexity Analysis
Let n be the maximum numbers of keys allowed in the pattern.

- **Time complexity: O(9⋅8^n)**

The method numberOfPatterns iterates through all 9 dots on the grid as a starting point.

In each call to countPatternsFromDot, the function explores all possible moves from the current dot. Let 8 be the approximate number of choices at each dot. In the worst-case scenario, each recursive call leads to further recursive calls, up to a maximum depth of n. Thus, the total number of patterns explored can be approximated by 9×8^n (each move branching out into multiple further moves).

Thus, the overall time complexity of the algorithm is O(9⋅8^n).

- **Space complexity: O(n)**

The arrays `SINGLE_STEP_MOVES` and `SKIP_DOT_MOVES` use constant space.

The visitedDots matrix is a 3×3 boolean array, which takes up constant space.

The maximum depth of the recursion stack is n.

Thus, the overall space complexity of the algorithm is O(n).

---

![image.png](https://assets.leetcode.com/users/images/56a4d835-57fb-4112-a0ef-9716cbac36b3_1723094847.720578.png)

### Algorithm
Main method `numberOfPatterns`:

![image.png](https://assets.leetcode.com/users/images/9cda2474-f55d-42cc-a07f-01e2d30ac45a_1723094914.5428002.png)


### Code
JAVA
```java []
class Solution {

    public int numberOfPatterns(int m, int n) {
        int[][] jump = new int[10][10];

        // Initialize the jump over numbers for all valid jumps
        jump[1][3] = jump[3][1] = 2;
        jump[4][6] = jump[6][4] = 5;
        jump[7][9] = jump[9][7] = 8;
        jump[1][7] = jump[7][1] = 4;
        jump[2][8] = jump[8][2] = 5;
        jump[3][9] = jump[9][3] = 6;
        jump[1][9] = jump[9][1] = jump[3][7] = jump[7][3] = 5;

        boolean[] visitedNumbers = new boolean[10];
        int totalPatterns = 0;

        // Count patterns starting from corner numbers (1, 3, 7, 9) and multiply by 4 due to symmetry
        totalPatterns +=
        countPatternsFromNumber(1, 1, m, n, jump, visitedNumbers) * 4;

        // Count patterns starting from edge numbers (2, 4, 6, 8) and multiply by 4 due to symmetry
        totalPatterns +=
        countPatternsFromNumber(2, 1, m, n, jump, visitedNumbers) * 4;

        // Count patterns starting from the center number (5)
        totalPatterns +=
        countPatternsFromNumber(5, 1, m, n, jump, visitedNumbers);

        return totalPatterns;
    }

    private int countPatternsFromNumber(
        int currentNumber,
        int currentLength,
        int minLength,
        int maxLength,
        int[][] jump,
        boolean[] visitedNumbers
    ) {
        // Base case: if current pattern length exceeds maxLength, stop exploring
        if (currentLength > maxLength) return 0;

        int validPatterns = 0;
        // If current pattern length is within the valid range, count it
        if (currentLength >= minLength) {
            validPatterns++;
        }

        visitedNumbers[currentNumber] = true;

        // Explore all possible next numbers
        for (int nextNumber = 1; nextNumber <= 9; nextNumber++) {
            int jumpOverNumber = jump[currentNumber][nextNumber];
            // Check if the next number is unvisited and either:
            // 1. There's no number to jump over, or
            // 2. The number to jump over has been visited
            if (
                !visitedNumbers[nextNumber] &&
                (jumpOverNumber == 0 || visitedNumbers[jumpOverNumber])
            ) {
                validPatterns +=
                countPatternsFromNumber(
                    nextNumber,
                    currentLength + 1,
                    minLength,
                    maxLength,
                    jump,
                    visitedNumbers
                );
            }
        }

        // Backtrack: unmark the current number before returning
        visitedNumbers[currentNumber] = false;

        return validPatterns;
    }
}
```

### Complexity Analysis
Let n be the maximum number of keys allowed in the pattern.

- **Time complexity: O(3⋅8^ n)**

The algorithm calls the recursive function countPatternsFromNumber a total of 3 times. Each recursive call explores approximately 8 surrounding dots in each call. In the worst case, each recursive call can spread up to a maximum depth of n with a branching factor of 8. Thus, the total time complexity of the algorithm comes out to be O(3⋅8^n).

- **Space complexity: O(n)**

The `jump` array is a 10×10 grid which takes constant space. The maximum depth of recursion can be n in the worst case. Thus, the overall space complexity is O(n).

### Approach 3: Memoization
### Intuition
As we recursively construct each pattern, we often encounter sub-problems that we've previously solved. For example, in the sequences [7, 8, 5] and [8, 7, 5], the number of patterns emerging from 5 will be identical in both cases, as it depends solely on the current number and the dots visited thus far. To avoid repeatedly calculating these overlapping sub-problems, we can significantly improve our time complexity by optimizing our algorithm.

This is where dynamic programming comes in. The essence of dynamic programming is to save (memoize) the results of previously computed sub-problems, so that if we encounter the same sub-problem again, we can directly return the saved result instead of recalculating it. But how do we uniquely identify a sub-problem? The answer lies in its state. Each recursive state is defined by two factors: the current number from which the pattern emerges, and the dots visited up to that point. This state uniquely identifies each sub-problem, allowing us to store the result in a dp table using this state as the identifier.

However, storing all visited numbers in a boolean array is cumbersome to use as an identifier. We need something simpler, like an integer value. Considering the constraints of the problem, the maximum number of dots possible is 9. We can use a 9-digit binary number, where each digit can be a 0 or a 1, to represent whether a number has been visited (1) or not (0). This 9-digit binary number effectively replaces the visitedNumbers array, allowing us to memoize the recursion results in a `dp` table of size 10×(1<<10).

To manipulate this new visitedNumbers integer, we need three essential functions:

1.`setBit:` Toggles the `ith` bit of the number to 1.

2.`clearBit:` Toggles the `ith`bit of the number to 0.

3.`isSet:` Checks whether the `ith` bit is 0 or 1.
For more information on bit manipulation concepts, you can refer to this LeetCode [Explore Card](https://leetcode.com/explore/learn/card/bit-manipulation/669/bit-manipulation-concepts/4496/).

![image.png](https://assets.leetcode.com/users/images/43cff49a-0ae1-4a63-84d5-b430f68cb600_1723095250.3600733.png)
![image.png](https://assets.leetcode.com/users/images/de20695f-30d9-4a3d-acfb-740b92d1e297_1723095304.5535223.png)
![image.png](https://assets.leetcode.com/users/images/0a0cc785-cfe5-4084-8023-276d99674713_1723095388.7496047.png)


### Code

JAVA

```Java []
class Solution {

    public int numberOfPatterns(int m, int n) {
        int[][] jump = new int[10][10];

        // Initialize the jump over numbers for all valid jumps
        jump[1][3] = jump[3][1] = 2;
        jump[4][6] = jump[6][4] = 5;
        jump[7][9] = jump[9][7] = 8;
        jump[1][7] = jump[7][1] = 4;
        jump[2][8] = jump[8][2] = 5;
        jump[3][9] = jump[9][3] = 6;
        jump[1][9] = jump[9][1] = jump[3][7] = jump[7][3] = 5;

        int visitedNumbers = 0;
        int totalPatterns = 0;
        Integer[][] dp = new Integer[10][1 << 10];

        // Count patterns starting from corner numbers (1, 3, 7, 9) and multiply by 4 due to symmetry
        totalPatterns +=
        countPatternsFromNumber(1, 1, m, n, jump, visitedNumbers, dp) * 4;

        // Count patterns starting from edge numbers (2, 4, 6, 8) and multiply by 4 due to symmetry
        totalPatterns +=
        countPatternsFromNumber(2, 1, m, n, jump, visitedNumbers, dp) * 4;

        // Count patterns starting from the center number (5)
        totalPatterns +=
        countPatternsFromNumber(5, 1, m, n, jump, visitedNumbers, dp);

        return totalPatterns;
    }

    private int countPatternsFromNumber(
        int currentNumber,
        int currentLength,
        int minLength,
        int maxLength,
        int[][] jump,
        int visitedNumbers,
        Integer[][] dp
    ) {
        // Base case: if current pattern length exceeds maxLength, stop exploring
        if (currentLength > maxLength) return 0;

        if (
            dp[currentNumber][visitedNumbers] != null
        ) return dp[currentNumber][visitedNumbers];

        int validPatterns = 0;
        // If current pattern length is within the valid range, count it
        if (currentLength >= minLength) {
            validPatterns++;
        }

        visitedNumbers = setBit(visitedNumbers, currentNumber);

        // Explore all possible next numbers
        for (int nextNumber = 1; nextNumber <= 9; nextNumber++) {
            int jumpOverNumber = jump[currentNumber][nextNumber];
            // Check if the next number is unvisited and either:
            // 1. There's no number to jump over, or
            // 2. The number to jump over has been visited
            if (
                !isSet(visitedNumbers, nextNumber) &&
                (jumpOverNumber == 0 || isSet(visitedNumbers, jumpOverNumber))
            ) {
                validPatterns +=
                countPatternsFromNumber(
                    nextNumber,
                    currentLength + 1,
                    minLength,
                    maxLength,
                    jump,
                    visitedNumbers,
                    dp
                );
            }
        }

        // Backtrack: unmark the current number before returning
        visitedNumbers = clearBit(visitedNumbers, currentNumber);

        return dp[currentNumber][visitedNumbers] = validPatterns;
    }

    private int setBit(int num, int position) {
        num |= 1 << (position - 1);
        return num;
    }

    private int clearBit(int num, int position) {
        num ^= 1 << (position - 1);
        return num;
    }

    private boolean isSet(int num, int position) {
        int bitAtPosition = (num >> (position - 1)) & 1;
        return bitAtPosition == 1;
    }
}
```

### Complexity Analysis
Let `n` be the maximum numbers of keys allowed in the pattern.

- **Time complexity: O(1)**

Due to memoization, the time complexity of the algorithm is bounded by the total time required to fill the dp array. The total size of dp is 10×(1<<10) or 10240. So, the overall time complexity of the algorithm is O(10240), which can be simplified to O(1).

- **Space complexity: O(n)**

The `jump` array and the dp array both use constant space irrelevant of the input. The recursion stack has a space complexity of O(n). Thus, the space complexity of the algorithm is O(n).